In [ ]:
# All the import statements we need for preprocessing the data 
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
import yfinance as yf
from sklearn.preprocessing import StandardScaler

# Setting a random seed
np.random.seed(42)

# Cleaning and reading the csv file 
df = pd.read_csv("../Data/^GSPC.csv")
df = df[df["Price"] != "Ticker"]
df = df[df["Price"] != "Date"]
df[['Close','High', 'Low', 'Open', 'Volume']] = df[['Close', 'High', 'Low', 'Open', 'Volume']].astype(float)
df['Price'] = pd.to_datetime(df['Price'])
df.set_index('Price', inplace=True)
df.index.name = None

# This function adds technical indicators and features to the data 
def add_features(data):
    df = data.copy()

    # These are the technical indicators that we will use from the yfinance library
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA10'] = df['Close'].rolling(window=10).mean()
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['Volatility'] = df['Close'].pct_change().rolling(window=10).std()
    df['Momentum'] = df['Close'] - df['Close'].shift(5)
    df['Return'] = df['Close'].pct_change()

    # These are features that we engineered from the technical indicators that we will use for our model. We added the 
    # X_ prefix to the features to indicate that they are engineered features
    df['X_MA5'] = (df['Close'] - df['MA5']) / df['Close']
    df['X_MA10'] = (df['Close'] - df['MA10']) / df['Close']
    df['X_MA20'] = (df['Close'] - df['MA20']) / df['Close']
    df['X_MA5_10'] = (df['MA5'] - df['MA10']) / df['Close']
    df['X_MA10_20'] = (df['MA10'] - df['MA20']) / df['Close']
    df['X_Volatility'] = df['Volatility']
    df['X_Momentum'] = df['Momentum']
    df['X_Return'] = df['Return']
    df['X_Return_5'] = df['Return'].rolling(5).sum()
    df['X_VOL_CHG'] = df['Volume'].pct_change(5)
    # The 'Target' column is a ternary classification problem with labels -1, 0, and 1 indicating down, neutral, and up movements
    df['Target'] = np.where(df['Return'].shift(-1) > 0.005, 1,
                    np.where(df['Return'].shift(-1) < -0.005, -1, 0))

    return df  

# The get_X and get_y functions are helper functions that extract features and labels from the data
def get_X(data):
    feature_columns = [col for col in data.columns if col.startswith('X_')]
    return data[feature_columns].values
def get_y(data):
    return data.Target.values

# We apply the features to the dataframe 
df = add_features(df)


/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

class ActorCritic(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_actions):
        super().__init__()
        self.shared_layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        
        # This is the Actor network (policy), which outputs action probabilities
        self.actor = nn.Sequential(
            nn.Linear(hidden_dim, num_actions),
            nn.Softmax(dim=-1)
        )
        
        # This is the Critic network, which outputs state value
        self.critic = nn.Sequential(
            nn.Linear(hidden_dim, 1)
        )
    # This function takes the input and passes it through the shared layers, then through the actor and critic networks
    def forward(self, x):
        shared_features = self.shared_layers(x)
        action_probs = self.actor(shared_features)
        state_value = self.critic(shared_features)
        return action_probs, state_value
    # This function samples an action from the policy and returns the action and its log probability
    def act(self, state):
        action_probs, _ = self.forward(state)
        m = Categorical(action_probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

class RLStrategy(Strategy):
    price_delta = 0.01
    lookback_window = 200
    hidden_dim = 64
    learning_rate = 0.001
    gamma = 0.99  # gamma is our discount factor, which determines how much we care about future rewards
    
    def init(self):
        # Feature + label prep
        full_df = add_features(self.data.df).fillna(method='ffill').fillna(method='bfill')
        self.all_data = full_df.loc[self.data.df.index.intersection(full_df.index)]
        self.features = [col for col in self.all_data.columns if col.startswith("X_")] # features
        self.input_dim = len(self.features) # number of features
        

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = ActorCritic(
            input_dim=self.input_dim,
            hidden_dim=self.hidden_dim,
            num_actions=3  # -1, 0, 1
        ).to(self.device) # initalize model
        
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate) # using the adam optimizer
        
        # RL specific variables
        self.saved_log_probs = []
        self.saved_state_values = []
        self.rewards = []
        self.prev_state = None
        self.prev_action = None
        self.prev_log_prob = None
        self.prev_state_value = None
        
        self.predictions = []
        self.actuals = []
        
    def calculate_reward(self, action, y_true):
        # A simple reward system where we have 1 for correct prediction, -1 for incorrect prediction 
        action_map = {0: -1, 1: 0, 2: 1}
        mapped_action = action_map[action]
        if mapped_action == y_true:
            return 1.0
        elif mapped_action == 0 or y_true == 0:  # Neutral cases
            return 0.0
        else:
            return -1.0
            
    def update_policy(self):
        # Fixed method to ensure tensors are the same size
        if len(self.saved_log_probs) == 0 or len(self.rewards) == 0:
            return
            
        # Ensure all lists are the same length by truncating
        min_length = min(len(self.saved_log_probs), len(self.saved_state_values), len(self.rewards))
        
        if min_length == 0:
            return
            
        # Convert lists to tensors
        log_probs = torch.stack(self.saved_log_probs[:min_length])
        state_values = torch.stack(self.saved_state_values[:min_length])
        rewards = torch.FloatTensor(self.rewards[:min_length]).to(self.device)
        
        # calculating returns
        returns = []
        R = 0
        for r in reversed(rewards): 
            R = r + self.gamma * R
            returns.insert(0, R)
        returns = torch.FloatTensor(returns).to(self.device)
        # This normalizes returns only if we have enough data
        if len(returns) > 1:
            returns = (returns - returns.mean()) / (returns.std() + 1e-8)
        advantages = returns - state_values # calculates advantages 
        
        # Calculate losses
        actor_loss = -(log_probs * advantages.detach()).mean()
        critic_loss = advantages.pow(2).mean()
        loss = actor_loss + critic_loss
        # This updates the network
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        # This clears memory for the next iteration 
        self.saved_log_probs = []
        self.saved_state_values = []
        self.rewards = []
    
    # This function is called at each step of the backtest
    def next(self):
        # Current position in the backtest
        current_idx = len(self.data) - 1
        
        # This safety check ensures we have enough data to make a prediction
        if current_idx < self.lookback_window:
            return
            
        # Get current data index
        current_candle_time = self.data.index[-1]
        
        try:
            # Find corresponding index in our all_data DataFrame
            idx_in_all_data = self.all_data.index.get_indexer([current_candle_time])[0]
            
            # Check if valid index found
            if idx_in_all_data < 0 or idx_in_all_data < self.lookback_window:
                return
                
            # Safe slicing with loc instead of iloc
            start_idx = self.all_data.index[idx_in_all_data - self.lookback_window + 1]
            end_idx = self.all_data.index[idx_in_all_data]
            history_data = self.all_data.loc[start_idx:end_idx]
            
            # Check if we have enough data
            if len(history_data) < self.lookback_window // 2:
                return
                
            # Current observation
            current_obs = history_data.iloc[-1][self.features].values
            current_y = history_data.iloc[-1]["Target"]
            
            # Convert to PyTorch tensor
            state = torch.FloatTensor(current_obs).to(self.device)
            
            # Get action from policy
            self.model.eval()
            with torch.no_grad():
                action_probs, state_value = self.model(state)
                m = Categorical(action_probs)
                action = m.sample().item()
                log_prob = m.log_prob(torch.tensor(action).to(self.device))
            
            # Map action to trading signal
            action_map = {0: -1, 1: 0, 2: 1}
            prediction = action_map[action]
            
            # If we have a previous state, calculate reward and store
            if self.prev_state is not None:
                reward = self.calculate_reward(self.prev_action, current_y)
                self.rewards.append(reward)
                self.saved_log_probs.append(self.prev_log_prob)
                self.saved_state_values.append(self.prev_state_value)
                
                # Periodically update policy, fixed to check list lengths first
                if len(self.rewards) >= 10:
                    self.update_policy()
            
            # The following four lines of code store the current state of the model for the next iteration
            self.prev_state = state
            self.prev_action = action
            self.prev_log_prob = log_prob
            self.prev_state_value = state_value.squeeze()
            
            # append the prediction and actual values to the lists 
            if prediction in [-1, 0, 1]:
                self.predictions.append(prediction)
                self.actuals.append(int(current_y))
                
            # Trade execution with take-profit and stop-loss
            close = self.data.Close[-1]
            tp_long = close * (1 + self.price_delta)
            sl_long = close * (1 - self.price_delta)
            tp_short = close * (1 - self.price_delta)
            sl_short = close * (1 + self.price_delta)
            
            # This portion is for the trading logic of our code
            if prediction == 1:
                if not self.position.is_long:
                    self.position.close()
                    self.buy(tp=tp_long, sl=sl_long)
                    
            elif prediction == -1:
                if not self.position.is_short:
                    self.position.close()
                    self.sell(tp=tp_short, sl=sl_short)
                    
            # This portion of the code is for risk management
            for trade in self.trades:
                if self.data.index[-1] - trade.entry_time > pd.Timedelta(days=5):
                    if trade.is_long:
                        trade.sl = max(trade.sl, trade.entry_price)
                    else:
                        trade.sl = min(trade.sl, trade.entry_price)
                        
        except (IndexError, KeyError, RuntimeError) as e:
            # This except statement is to catch any errors that may occur during the process
            # and to ensure that the backtest continues running smoothly
            pass

In [6]:
bt = Backtest(df, RLStrategy, cash=10_000, commission=.0002)
backtest = bt.run()

/var/folders/bg/6x82p8q55dxf8pb_c34j03mm0000gn/T/ipykernel_37849/1885506390.py:50: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  full_df = add_features(self.data.df).fillna(method='ffill').fillna(method='bfill')


Backtest.run:   0%|          | 0/986 [00:00<?, ?bar/s]

In [7]:
backtest

Start                     2020-01-30 00:00:00
End                       2023-12-29 00:00:00
Duration                   1429 days 00:00:00
Exposure Time [%]                     1.01317
Equity Final [$]                   9826.08453
Equity Peak [$]                   10075.56462
Commissions [$]                        22.993
Return [%]                           -1.73915
Buy & Hold Return [%]                45.25956
Return (Ann.) [%]                    -0.44694
Volatility (Ann.) [%]                 0.71298
CAGR [%]                             -0.30892
Sharpe Ratio                         -0.62687
Sortino Ratio                        -0.67962
Calmar Ratio                          -0.1805
Alpha [%]                            -1.79993
Beta                                  0.00134
Max. Drawdown [%]                    -2.47609
Avg. Drawdown [%]                    -2.47609
Max. Drawdown Duration     1138 days 00:00:00
Avg. Drawdown Duration     1138 days 00:00:00
# Trades                          

In [8]:
bt.plot()

GridPlot(id='p1327', ...)